In [0]:
CREATE TABLE IF NOT EXISTS nan_demo.default.dup_res (
  doc_id_1 STRING,
  doc_id_2 STRING,
  ai_result STRING
);

MERGE INTO nan_demo.default.dup_res AS target
USING (
  SELECT 
    a.path AS doc_id_1,
    b.path AS doc_id_2,
    ai_query(
      'databricks-meta-llama-3-3-70b-instruct',
      "Given the following two receipts, determine if these documents are duplicates of each other. Note: The receipt text may not be 100% accurate or identical due to extraction from PDF, so account for possible minor differences. Consider order items, sales amounts, restaurant name, and date/time. Respond only with 'Yes' or 'No'. Receipt 1 details: " || a.text || " Receipt 2 details: " || b.text
    ) AS ai_result
  FROM nan_demo.default.pdf_parsed a
  JOIN nan_demo.default.pdf_parsed b
      ON a.path < b.path
) AS source
ON target.doc_id_1 = source.doc_id_1 AND target.doc_id_2 = source.doc_id_2
WHEN MATCHED THEN
  UPDATE SET ai_result = source.ai_result
WHEN NOT MATCHED THEN
  INSERT (doc_id_1, doc_id_2, ai_result)
  VALUES (source.doc_id_1, source.doc_id_2, source.ai_result)